<a href="https://colab.research.google.com/github/jhonbot/jhonbot.github.io/blob/main/Rclone_file_transfer_in_colab_with_JD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<CENTER><br><img src='https://goodvpnservice.com/wp-content/uploads/2017/06/illegal-downloading.jpg' height="25" alt="Gdrive-logo"/> <font size=6px >Download And Extraction By</font> <img src='https://imagizer.imageshack.com/img923/5319/PcnbHe.png'height="" alt="Gdrive-logo"/>






In [ ]:
#@title <<----<strong>Install rClone</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
RCLONE = True #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True) 
  
if RCLONE==True:
  !curl https://rclone.org/install.sh | sudo bash

if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button"); 
      btn.click() 
      }
    
    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect"); 
      btn.click() 
      }
    }
    
  setInterval(ClickConnect,60000)
  '''))

In [ ]:
#@title <-----<strong>Upload your  Rclone.config file</strong>
def moveConfig():
  !mv rclone.conf /root/.config/rclone/rclone.conf

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
moveConfig()
print("Moved rclone.conf to /root/.config/rclone/rclone.conf")

Saving rclone.conf to rclone.conf
User uploaded file "rclone.conf" with length 622 bytes
Moved rclone.conf to /root/.config/rclone/rclone.conf


# Rclone Mount/Unmount/Copy </center>
<img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>

In [ ]:
#@markdown <h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
##########################################

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))
content = open("/root/.config/rclone/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)
cache_path="/content/temp/rCloneTemp"
def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    !rclone mount $mountNam.value: $mPoint --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir '/content/temp/rCloneTemp' --allow-other --daemon 

    if os.path.isdir(mPoint)== True:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

Mount success! - 	/content/drives/jdownloaderhero


In [ ]:
 
#@markdown <img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>
 
#@markdown ---
source = "/root/Downloads/sample.pdf" #@param {type:"string"}
destination = "/content/drives/jdownloaderhero/Bot_Uploader/Uncharted" #@param {type:"string"}
mode = "copy" #@param ["copy", "check"]
dry_run = False #@param {type:"boolean"}
Extra_args = "" #@param {type:"string"}
#@markdown ---
Email_notification = False #@param {type:"boolean"}
logs = False #@param {type:"boolean"}
emailID = "" #@param {type:"string"}
password = "" #@param {type:"string"}
Receiver_ID = "" #@param {type:"string"}
#@markdown ---
import smtplib 
########################################
args = "--transfers 20 --checkers 20 --stats-one-line --stats=5s -v --tpslimit 95 --tpslimit-burst 40 "
if mode == "check":
  args += "--one-way "
if dry_run == True:
    args +=" --dry-run "
if logs == True:
  args += " --log-file rClone_log.txt "
if Extra_args != "":
  args +=Extra_args
######################################
def runrClone():
  !rclone --user-agent "Mozilla" "$mode" "$source" "$destination" $args
  
def checkEmail():
  if (Email_notification == True and logs == False):
    print("You will receive only a notification after task has finished.")
    runrClone()
    #Send only notification
     
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(emailID , password)
    message = "Your rClone task has Completed!"
    s.sendmail(emailID, Receiver_ID, message)
    print("Email Alert Sent!")  
    s.quit()
  elif (Email_notification == True and logs == True):
    print("You will receive a notification with log attached after task has finished.")
    runrClone()
    #Sending email notification with logs
    
    from email.mime.multipart import MIMEMultipart 
    from email.mime.text import MIMEText 
    from email.mime.base import MIMEBase 
    from email import encoders 

    fromaddr = emailID
    toaddr = Receiver_ID
    msg = MIMEMultipart() 
    # storing the senders email address 
    msg['From'] = fromaddr 
    # storing the receivers email address 
    msg['To'] = toaddr 
    # storing the subject 
    msg['Subject'] = "Colab has Finished Running your Cell"
    # string to store the body of the mail 
    body = "Your rClone Task has Completed! Visit link to view : https://colab.research.google.com/github/SKGHD/Handy/blob/master/HandBrakeCLI_Colab.ipynb"

    msg.attach(MIMEText(body, 'plain'))
    filename = "rClone_log.txt"
    attachment = open("/content/rClone_log.txt", "rb")
    p = MIMEBase('application', 'octet-stream') 
    p.set_payload((attachment).read()) 
    # encode into base64 
    encoders.encode_base64(p) 
 
    p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
    msg.attach(p)  
    s = smtplib.SMTP('smtp.gmail.com', 587) 
    s.starttls() 
    s.login(fromaddr, password) 
    text = msg.as_string() 
    s.sendmail(fromaddr, toaddr, text)
    print("Email Alert Sent with log!")  
    s.quit() 
  elif (Email_notification == False and logs == False):
    print("You will not receive any notification!!!.")
    runrClone()
checkEmail()

---
# <img src='https://szyha.github.io/RcloneLabArchive/img/title_jdownloader.png' height="45" alt="JDownloader"/>

In [ ]:
#@markdown <h3>⬅️ Clicca qui per installare JDownloader</h3>

import os, uuid, re, IPython
import ipywidgets as widgets

from glob import glob
from IPython.display import HTML, clear_output
from google.colab import output

Email = widgets.Text(placeholder="*Required", description="Email:")
Password = widgets.Text(placeholder="*Required", description="Password:")
Device = widgets.Text(placeholder="Optional", description="Name:")
SavePath = widgets.Text(placeholder="/content/Downloads", description="Save Path:")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPath():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePath.options = ["/content", "/content/Downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePath.options = ["/content", "/content/Downloads"]

def LoginForm():
  clear_output(wait=True)
  Email.value = ""
  Password.value = ""
  Device.value = ""
  RefreshPath()
  display(HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If you don't have an account yet, please register <a href=\"https://my.jdownloader.org/login.html#register\" target=\"_blank\">here</a>.</h3>"), HTML("<br>"), Email, Password, Device, SavePath, MakeButton("Refresh", RefreshPath, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML(""))
  display(HTML("<br>"), MakeButton("Login", CheckLogin, "info"))
  if os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
    display(MakeButton("Cancel", Show, "danger"))
  
def RestartForm():
  clear_output(wait=True)
  display(MakeLabel("Restart Confirm?", ""), MakeButton("Confirm", Restart, "danger"), MakeButton("Cancel", Show, "warning"))
  
def ExitForm():
  clear_output(wait=True)
  display(MakeLabel("Exit Confirm?", ""), MakeButton("Confirm", Exit, "danger"), MakeButton("Cancel", Show, "warning"))

def CheckLogin():
  try:
    if not Email.value.strip():
      ERROR = "Email field is empty."
      THROW_ERROR
    if not "@" in Email.value and not "." in Email.value:
      ERROR = "Email is an incorrect format."
      THROW_ERROR
    if not Password.value.strip():
      ERROR = "Password field is empty."
      THROW_ERROR
    if not bool(re.match("^[a-zA-Z0-9]+$", Device.value)) and Device.value.strip():
      ERROR = "Only alphanumeric are allowed for the device name."
      THROW_ERROR
    Login()
  except:
    print(ERROR)

def Login():
  clear_output(wait=True)
  if SavePath.value == "/content":
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  elif SavePath.value == "/content/Downloads":
    get_ipython().system_raw("mkdir -p -m 666 /content/Downloads")
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"/content/Downloads\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  else:
    get_ipython().system_raw("echo '{\"defaultdownloadfolder\" : \"" + SavePath.value + "\"}' > /root/.JDownloader/cfg/org.jdownloader.settings.GeneralSettings.json")
  if Device.value.strip() == "":
    Device.value = Email.value
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("echo '{\"email\" : \"'" + Email.value + "'\", \"password\" : \"'" + Password.value + "'\", \"devicename\" : \"'" + Device.value + "'\", \"directconnectmode\" : \"LAN\"}' > /root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()

def Restart():
  get_ipython().system_raw("pkill -9 -e -f java")
  get_ipython().system_raw("java -jar /root/.JDownloader/JDownloader.jar -norestart -noerr -r &")
  Show()
  
def Exit():
  get_ipython().system_raw("pkill -9 -e -f java")
  clear_output(wait=True)
  display(MakeButton("Start", Restart, "info"))
  
def Show():
  clear_output(wait=True)
  display(MakeLabel("Control Panel", ""), HTML("<h3 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">You can login to the WebUI by clicking <a href=\"https://my.jdownloader.org/\" target=\"_blank\">here</a>.</h3>"), HTML("<h4 style=\"font-family:Trebuchet MS;color:#4f8bd6;\">If the server didn't showup in 30 sec. please re-login.</h4>"), HTML("<br>"), MakeButton("Re-Login", LoginForm, "info"), MakeButton("Restart", RestartForm, "warning"), MakeButton("Exit", ExitForm, "danger"))
  
    
if not os.path.isfile("/root/.JDownloader/JDownloader.jar"):
  clear_output(wait=True)
  display(MakeLabel("Installing in Progress", "warning"))
  get_ipython().system_raw("rm -rf /content/sample_data/ && apt install openjdk-11-jre-headless -qq -y && mkdir -p -m 666 /root/.JDownloader && wget -q http://installer.jdownloader.org/JDownloader.jar -O /root/.JDownloader/JDownloader.jar && java -jar /root/.JDownloader/JDownloader.jar -norestart -h")
  LoginForm()
elif not os.path.isfile("/root/.JDownloader/cfg/org.jdownloader.api.myjdownloader.MyJDownloaderSettings.json"):
  LoginForm()
else:  
  Show()

---
# <img src='https://github.com/jhonbot/jhonbot.github.io/raw/main/Image/WinRAR-Logo.jpg' height="65"/>

In [ ]:
Modalità = 'Estrazione con o senza password' #@param ["Estrazione con o senza password"]
# @markdown **⚠️Se l'archivio è protetto, inserisci la password, altrimenti schiaccia INVIO e lascia vuoto il campo password⚠️**
percorso_cartella_rar = "/content/drive/Shareddrives/JhonStream/Bot_Uploader/Uncharted/Uncharted.2022.2160p.DV.WEB-DL.DD5.1.H.265-DDN.part01.rar" #@param {type:"string"}
cloud_path = "/content/drive/Shareddrives/JhonStream/Bot_Uploader" #@param {type:"string"}

!unrar x -P "" -n {percorso_cartella_rar} {cloud_path}


Enter password (will not be echoed): 


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Enter password (will not be echoed) for /content/drive/Shareddrives/JhonStream/Bot_Uploader/Uncharted/Uncharted.2022.2160p.DV.WEB-DL.DD5.1.H.265-DDN.part01.rar: 

Program aborted


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt install unrar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
percorso_file_rar = "/content/drives/jdownloaderhero/Downloads/Diabolik" #@param {type:"string"}
!sudo unrar l -p "" {percorso_file_rar}


In [ ]:
!sudo unrar x -p "" -n "/content/drives/jdownloaderhero/Downloads/Diabolik/Diabolik (2021) FullHD 1080p VU ITA DTS HD MA AC3 Sub.part01.rar" "Diabolik (2021) FullHD 1080p VU ITA DTS HD MA AC3 Sub.mkv" "/content/drives/jdownloaderhero/Downloads"